In [1]:
import numpy as np
import matplotlib.pyplot as plt
from Potential import Potential
from Metode.GreenFunction import GreenFunction_V2
from Metode.AnalyticalMethod import AnalyticalMethod_V2
from Metode.MatrixTransfer import TransferMatrixMethod_V2
N  = 100
a = 6
m = 1
h = 1
E = 8
V_max = 10

get = Potential(N, a,E, m, h)
x = get.x
potential_barrier = get.PotentialBarrier(get,V_max,0)
step_potential = get.StepPotential(get,V_max,0)
morse_feshbach_potential_well = get.MorseFeshbach(get,V_max,1,0,0.2)

In [2]:
t = h**2/(2*m)*np.ones(N+1)
V1 = potential_barrier.get_potential()
from scipy.sparse import diags
T = diags([-t[:-1], V1+2*t, -t[:-1]], [-1, 0, 1], shape=(N,N)).toarray()

In [3]:
from numpy import ones,imag,pi, matrix
from scipy.sparse import diags, eye
from scipy.sparse.linalg import inv as inverse


class GreenFunction_V3:
    def __init__(self, m, h, E):
        __name__ = "Metode Green's Function "
        self.m = m
        self.h = h
        self.E = E
        self.t = self.h**2/(2*self.m)
    
    def Hamiltonian(self, V):
        #N= V.parent.N
        dx= V.parent.x[1]-V.parent.x[0]
        self.t = self.t/dx
        N=3
        V = V.get_potential()
        V = np.array([V[0],V[50],V[-1]])
        t = self.t*ones(N+1)
        #H = diags([-t[:-1], V+2*t, -t[:-1]], [-1, 0, 1], shape=(N,N))
        H = diags([-t[:-1], V+2*t[:-1], -t[:-1]], [-1, 0, 1], shape=(N,N))
        return H
    
    def GreenFunction_for_finding_Self_Energy(self, V, Green_surfaceL=0,Green_surfaceR=0):
        self.Green_surfaceL = Green_surfaceL
        self.Green_surfaceR = Green_surfaceR
        #N = V.parent.N
        dx= V.parent.x[1]-V.parent.x[0]
        self.t = self.t/dx
        N=3
        H = self.Hamiltonian(V)
        Sigma=eye(N) * 0
        Sigma=Sigma.tocsc()
        Sigma[0,0],Sigma[-1,-1] = self.t*Green_surfaceL*self.t, self.t*Green_surfaceR*self.t
        G = inverse(self.E*eye(N) - H -Sigma)                     
        return G
    
    def GreenFunction_for_finding_center_green_function(self, V):
        GreenFunction_0 = self.GreenFunction_for_finding_Self_Energy(V,0,0)
        gL,gR = GreenFunction_0[0,0],GreenFunction_0[-1,-1]
        Gc = self.GreenFunction_for_finding_Self_Energy(V,gL,gR)
        Gc = Gc[Gc.shape[0] // 2, Gc.shape[1] // 2]
        return Gc,gL,gR
    
    def DensityOfState_in_center(self, V):
        G = self.GreenFunction_for_finding_center_green_function(V)
        DOS = -1/pi*imag(G)
        return DOS
    
    #full green's function, masih belum yakin benar ??
    def GreenFunction(self, V):
        Gc,gL,gR = self.GreenFunction_for_finding_center_green_function(V)
        GL,GCL,GRL = gL*(1+self.t*Gc*self.t*gL), gL*self.t*Gc, gL*self.t*Gc*self.t*gR
        GLC,GC,GRC = Gc*self.t*gL, Gc, Gc*self.t*gR
        GLR,GCR,GR = gR*self.t*Gc*self.t*gL, gR*self.t*Gc, gR*(1+self.t*Gc*self.t*gR)
        G  = matrix([[ GL,GCL,GRL ],
                     [ GLC,GC,GRC ],
                     [ GLR,GCR,GR ]])
        return G
    
    def psi():
        pass

In [4]:

Green2 = GreenFunction_V3(m=1,h=1,E=E)
H = Green2.Hamiltonian(potential_barrier).toarray()
G_0 = Green2.GreenFunction_for_finding_Self_Energy(potential_barrier)
G_C = Green2.GreenFunction_for_finding_center_green_function(potential_barrier)
DOS = Green2.DensityOfState_in_center(potential_barrier)

G_full = Green2.GreenFunction(potential_barrier)

/home/codespace/.local/lib/python3.12/site-packages/scipy/sparse/_index.py:108: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/home/codespace/.local/lib/python3.12/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:603: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/home/codespace/.local/lib/python3.12/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


In [5]:
from Potential import Potential, intial_wavefunction


Psi_0 = intial_wavefunction(N,a,E).gasussian_wavepacket(potential_barrier,-2,.5)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class AnalyticalMethod_3:
    def __init__(self, V):
        m = V.parent.m
        h = V.parent.h
        k = V.get_wavevector()
        dx= V.parent.x[1]-V.parent.x[0]
        self.t = h**2/(2*m)
        self.t = self.t/self.dx
        N=V.parent.N 

    def Hamiltonian(self, V):
        V = V.get_potential()
        t = t*ones(N+1)
        H = diags([-t[:-1], V+2*t, -t[:-1]], [-1, 0, 1], shape=(N,N))
        return H


In [ ]:
N,a,E,m,h,x=potential_barrier.parent.N

TypeError: cannot unpack non-iterable Potential object